# Getting COT data

### Imports

In [1]:
import pandas as pd
from datetime import date
from index import cot_all, cot_hist, cot_year, cot_description

### Historical COT till 2016 

In [2]:
his_df = cot_hist()

Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the directory data


### COT from 2017 till now

In [3]:
l = []

for i in range(2017, date.today().year+1):
    l.append(cot_year(i))

df = pd.concat(l)

Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data
Selected: legacy_fut
Stored the extracted file annual.txt in the directory data


### All COT data

In [5]:
all_df = pd.concat([his_df, df])

all_df = all_df.rename(columns={'As of Date in Form YYYY-MM-DD': 'Date'})

all_df['Date'] = pd.to_datetime(all_df['Date'])

all_df.set_index('Date', inplace=True)

all_df.sort_index(inplace=True)

all_df.head()

Market and Exchange Names  \
Date                                                            
1986-01-15  CRUDE OIL, LIGHT 'SWEET' - NEW YORK MERCANTILE...   
1986-01-15                      CORN - CHICAGO BOARD OF TRADE   
1986-01-15          LIVE CATTLE - CHICAGO MERCANTILE EXCHANGE   
1986-01-15              SOYBEAN MEAL - CHICAGO BOARD OF TRADE   
1986-01-15  PROPANE GAS - PETROLEUM ASSOC OF N Y COTTON EXCH.   

            As of Date in Form YYMMDD CFTC Contract Market Code  \
Date                                                              
1986-01-15                     860115                    067651   
1986-01-15                     860115                    002601   
1986-01-15                     860115                    057642   
1986-01-15                     860115                    026603   
1986-01-15                     860115                    066752   

           CFTC Market Code in Initials  CFTC Region Code  \
Date                                                        
1986-01-15                         NYME                 1   
1986-01-15                          CBT                 0   
1986-01-15                          CME                 0   
1986-01-15                          CBT                 0   
1986-01-15                         PANY                 1   

            CFTC Commodity Code  Open Interest (All)  \
Date                                                   
1986-01-15                   67                74334   
1986-01-15                    2               601935   
1986-01-15                   57                58475   
1986-01-15                   26                46028   
1986-01-15                   66                  303   

            Noncommercial Positions-Long (All)  \
Date                                             
1986-01-15                                2560   
1986-01-15                               45615   
1986-01-15                                3475   
1986-01-15                                7292   
1986-01-15                                   0   

            Noncommercial Positions-Short (All)  \
Date                                              
1986-01-15                                11594   
1986-01-15                                16565   
1986-01-15                                 4915   
1986-01-15                                  689   
1986-01-15                                    0   

            Noncommercial Positions-Spreading (All)  ...  \
Date                                                 ...   
1986-01-15                                     5317  ...   
1986-01-15                                    28015  ...   
1986-01-15                                     3842  ...   
1986-01-15                                     1780  ...   
1986-01-15                                        0  ...   

            Concentration-Gross LT =8 TDR-Long (Other)  \
Date                                                     
1986-01-15                                         0.0   
1986-01-15                                        46.7   
1986-01-15                                         0.0   
1986-01-15                                        27.2   
1986-01-15                                         0.0   

            Concentration-Gross LT =8 TDR-Short(Other)  \
Date                                                     
1986-01-15                                        69.8   
1986-01-15                                        21.6   
1986-01-15                                         0.0   
1986-01-15                                        47.5   
1986-01-15                                         0.0   

            Concentration-Net LT =4 TDR-Long (Other)  \
Date                                                   
1986-01-15                                       0.0   
1986-01-15                                      35.3   
1986-01-15                                       0.0   
1986-01-15                                      16.6   
1986-01-1

## Filter the asset

In [ ]:
ASSET_CODE = '088691'

In [ ]:
df[(df['Open Interest (Other)']!= 0) & (df['CFTC Contract Market Code'] == ASSET_CODE)]

In [ ]:
asset_df = all_df[all_df['CFTC Contract Market Code'] == ASSET_CODE]

## COT index

In [28]:
commercials_df = asset_df[['Commercial Positions-Long (All)', 'Commercial Positions-Short (All)']].copy()

commercials_df.columns = ['Long', 'Short']

commercials_df['Net Position'] = commercials_df['Long'] - commercials_df['Short']

commercials_df

Long   Short  Net Position
Date                                   
1986-01-15  80962   89990         -9028
1986-01-31  82262   85250         -2988
1986-02-14  94830   76735         18095
1986-02-28  94126   76200         17926
1986-03-14  85925   78653          7272
...           ...     ...           ...
2025-06-17  71684  309411       -237727
2025-06-24  73323  303883       -230560
2025-07-01  68820  304889       -236069
2025-07-08  71875  310229       -238354
2025-07-15  75989  326677       -250688

[1873 rows x 3 columns]

## COT Index Formula

The Commitments of Traders (COT) Index is calculated as:

$$
\text{COT Index} = \left( \frac{P_{\text{current}} - P_{\min}}{P_{\max} - P_{\min}} \right) \times 100
$$

Where:

$ P_{\text{current}} $: Current Net Position <br>
$ P_{\min} $: Lowest Net Position in the past  N  period <br>
$ P_{\max} $: Highest Net Position in the past  N  periods <br>
$ N $: Number of periods used in the lookback window


In [37]:
window = f"{365}D"  # or whatever number of periods you want

commercials_df['Max_Net_Position_Last_N'] = commercials_df['Net Position'].rolling(window=window).max()

commercials_df['Min_Net_Position_Last_N'] = commercials_df['Net Position'].rolling(window=window).min()

# commercials_df.dropna(inplace=True)

commercials_df['Index'] = 100 * (commercials_df['Net Position'] - commercials_df['Min_Net_Position_Last_N'])/(commercials_df['Max_Net_Position_Last_N'] - commercials_df['Min_Net_Position_Last_N'])

commercials_df

Long   Short  Net Position  Max_Net_Position_Last_N  \
Date                                                               
1986-01-15  80962   89990         -9028                  -9028.0   
1986-01-31  82262   85250         -2988                  -2988.0   
1986-02-14  94830   76735         18095                  18095.0   
1986-02-28  94126   76200         17926                  18095.0   
1986-03-14  85925   78653          7272                  18095.0   
...           ...     ...           ...                      ...   
2025-06-17  71684  309411       -237727                -190761.0   
2025-06-24  73323  303883       -230560                -190761.0   
2025-07-01  68820  304889       -236069                -190761.0   
2025-07-08  71875  310229       -238354                -190761.0   
2025-07-15  75989  326677       -250688                -190761.0   

            Min_Net_Position_Last_N       Index  
Date                                             
1986-01-15                  -9028.0         NaN  
1986-01-31                  -9028.0  100.000000  
1986-02-14                  -9028.0  100.000000  
1986-02-28                  -9028.0   99.376913  
1986-03-14                  -9028.0   60.096597  
...                             ...         ...  
2025-06-17                -339706.0   68.467555  
2025-06-24                -339706.0   73.279398  
2025-07-01                -339706.0   69.580718  
2025-07-08                -339706.0   68.046594  
2025-07-15                -339706.0   59.765685  

[1873 rows x 6 columns]

### Plotting COT index

In [44]:
import plotly.express as px

fig = px.line(commercials_df.reset_index(), x='Date', y='Index', title='COT index', markers=True, template='plotly_dark')

# Add green shaded area (e.g. for high index range)
fig.add_shape(
    type='rect',
    xref='paper',  # entire x-axis
    yref='y',
    x0=0,
    x1=1,
    y0=80,
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

# Add red shaded area (e.g. for low index range)
fig.add_shape(
    type='rect',
    xref='paper',
    yref='y',
    x0=0,
    x1=1,
    y0=0,
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.write_html("plots/commercials_index.html")
fig.show()


## Open Interest

In [9]:
oi_df = asset_df[['Open Interest (All)']].copy()

oi_df.columns = ['OI']

oi_df

OI
Date              
1986-01-15  148186
1986-01-31  144149
1986-02-14  139995
1986-02-28  143752
1986-03-14  142868
...            ...
2025-06-17  441214
2025-06-24  434958
2025-07-01  437662
2025-07-08  443144
2025-07-15  448531

[1873 rows x 1 columns]

## Net positions

> Commercial (Long) = Producer/Merchant/Processor/User(Long) + Swap Dealers(Long) + Swap Dealers(Spreading)

> Commercial (Short) = Producer/Merchant/Processor/User(Short) + Swap Dealers(Short) + Swap Dealers(Spreading)

> Non-Commercial (Long) = Managed Money(Long) + Other Reportables(Long) 

> Non-Commercial (Short) = Managed Money(Short) + Other Reportables(Short) 

> Non-Commercial (Spreading) = Managed Money(Spreading) + Other Reportables(Spreading) 

In [16]:
net_positions_df = pd.DataFrame()

net_positions_df['Commercials'] = commercials_df['Net Position']
net_positions_df['Large speculators'] = asset_df['Noncommercial Positions-Long (All)'] - asset_df['Noncommercial Positions-Short (All)']
net_positions_df['Small Traders'] = asset_df['Nonreportable Positions-Long (All)'] - asset_df['Nonreportable Positions-Short (All)']

net_positions_df

Commercials  Large speculators  Small Traders
Date                                                     
1986-01-15        -9028              10883          -1855
1986-01-31        -2988               6061          -3073
1986-02-14        18095              -6772         -11323
1986-02-28        17926              -4231         -13695
1986-03-14         7272               1622          -8894
...                 ...                ...            ...
2025-06-17      -237727             200648          37079
2025-06-24      -230560             195004          35556
2025-07-01      -236069             201980          34089
2025-07-08      -238354             202968          35386
2025-07-15      -250688             213115          37573

[1873 rows x 3 columns]

### Plotting

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for col in net_positions_df.columns:
    fig.add_trace(go.Scatter(x=net_positions_df.index, y=net_positions_df[col], mode='lines', name=col))

fig.update_layout(title='Net positions', xaxis_title='Date', yaxis_title='Contratcs', template='plotly_dark')

# fig.write_html("plots/net_positions.html")
fig.show()

## Plotting all charts


In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 1 column, 3 rows, shared X-axis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing= 0.1, subplot_titles=('COT Index', 'Net positions', 'OI'))

# Add each series to its own subplot
fig.add_trace(go.Scatter(x=asset_df.index, y=commercials_df['Index'], name='COT Index'), row=1, col=1)
for col in net_positions_df.columns:
    fig.add_trace(go.Scatter(x=asset_df.index, y=net_positions_df[col], name=col), row=2, col=1)
fig.add_trace(go.Scatter(x=asset_df.index, y=oi_df['OI'], name='OI'), row=3, col=1)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=80,              # Adjust based on your y-axis range
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=0,              # Adjust based on your y-axis range
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)


# Update layout
fig.update_layout(
    height=900,
    width=1450,
    title_text='Multiple Time Series with Shared X-Axis',
    template='plotly_dark',
    hovermode='x unified',  # or 'x unified' or 'closest'
)
fig.update_xaxes(title_text='Date', row=3, col=1)  # Only show x-axis label on bottom chart
fig.update_yaxes(title_text='CONTRACT')

# Save as responsive HTML
fig.write_html("plots/cot_analysis.html", auto_open=False, full_html=True, include_plotlyjs='cdn', config={'responsive': True})
fig.show()